In [1]:
# NVD and MITRE Feed Handler...
#!python -m spacy download en_core_web_sm
#!python -m spacy download en_core_web_lg

# Imports
import json, csv, os, requests, re, zipfile, spacy, timeit
import pandas as pd  
from os import listdir
from os.path import isfile, join
from spacy.matcher import Matcher
from datetime import datetime

# global variables and other classes
json_files_path, analysis_path, summary_csv, db_name = 'nvd_json_files', 'analysis', 'cyvia_dataset.csv', 'cyvia_dataset'
mitre_csv_path = 'mitre_csv_files/cwe_combined.csv'
cwe_data = pd.read_csv(mitre_csv_path) # load cwe data file in dataframs
summarized_dataset = analysis_path + '/' + summary_csv
enc = 'utf-8' # enc = 'utf-16', enc = 'iso-8859-15', enc = 'cp437'

import functions as fn # functions class
func = fn.functions()
import Spinner as sp # spinner while performing db operations
spinner = sp.Spinner()

# Prepare data from downloaded JSON files...
# This step reads the JSON files and writes the collected data to a CSV file and database. 
def read_and_summarize():
    print('Reading, summarizing, and writing NVD JSON Data...')
    column_names = ['_id','lang', 'cwe_id', 'cwe_desc', 'cwe_plat', 'cwe_af_res', 'cwe_consequences', 'cwe_mitigations', 'severity', 'cvss_v2', 'cvss_v3', 'vul_access_vector', 'user_int_req', 'os', 'sw', 'ports', 'published_date','modified_date','description', 'url_and_tags']

    # connect database
    func.connect_db(db_name, True) # create a new database
    # Start writing data...    
    with open(summarized_dataset, 'w', newline='') as file:
        writer = csv.writer(file)
        # write the header...
        writer.writerow(column_names)
        # data field variables as above line
        CVE_ID, CVE_Lang, CWE_ID, CWE_Desc, CWE_Plat, CWE_Af_Res, CVE_Sev, CVSS_V2, CVSS_V3, CVE_VAV, CVE_UIR, CVE_PD, CVE_MD, CVE_Desc = "", "", "", "", "", "", "", "", "", "", "", "", "", ""
        CWE_Consequences, CWE_Mitigations = {}, {} # Dictionaries
        CVE_OS, CVE_SW, CVE_Ports = [], [], [] # Lists
        # missing data field counters
        M_CVE_Lang, M_CWE_ID, M_CWE_ID_Other, M_CWE_ID_Other2, M_CWE_Desc, M_CWE_Plat, M_CWE_Af_Res, M_CVE_Sev, M_CVSS_V2, M_CVSS_V3, M_CVE_VAV, M_CVE_UIR, M_OS, M_SW, M_Ports, M_CVE_PD, M_CVE_MD, unicode_error, M_CWE_Mitigations, M_CWE_Consequences = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        vul_info_counter=0 # total number of handled vulnerabilities
        
        #Loop through input files in the directory...
        for root, dirs, files in os.walk(json_files_path + '/', topdown=False):
            for name in files:
                with open(os.path.join(root, name), 'r', encoding=enc) as f:
                    data=json.load(f)

                #Write json file data as rows to the new file...
                for i in range(0, len(data['CVE_Items'])):
                    vul_info_counter+=1 # increment vul counter
                    # extract URL and Tag references 
                    url_tag_dict = {}
                    # collecting references, can be more than one.
                    for x in range(0, len(data['CVE_Items'][i]['cve']['references']['reference_data'])): # first item till the last item.
                        url_tag_dict['URL'+str(x)] = data['CVE_Items'][i]['cve']['references']['reference_data'][x]['url'] # append dict.
                        # for tag list, add more than one tags.
                        for y in range(0, len(data['CVE_Items'][i]['cve']['references']['reference_data'][x]['tags'])): 
                            url_tag_dict['URL'+str(x)+'-Tag'+str(y)] = data['CVE_Items'][i]['cve']['references']['reference_data'][x]['tags'][y]             

                    # Fetching field[i]values from JSON file
                    # Field: CVE_ID, CVE-Description, available for all entries.
                    CVE_ID = data['CVE_Items'][i]['cve']['CVE_data_meta']['ID']
                    CVE_Desc = data['CVE_Items'][i]['cve']['description']['description_data'][0]['value']
                    # Extract keywords from the above description
                    CVE_OS, CVE_SW, CVE_Ports = [], [], []
                    #CVE_OS, CVE_SW, CVE_Ports = fetch_keywords(str(CVE_Desc)) # ** Enable this after testing

                    try: # Field: CVE-Language
                        CVE_Lang = data['CVE_Items'][i]['cve']['problemtype']['problemtype_data'][0]['description'][0]['lang'] #CVE Language
                    except IndexError: # Some CVEs are missing Vuln Type and Lang values.
                        CVE_Lang = "N/A"
                        M_CVE_Lang+=1

                    try: # Field: CWE-ID
                        CWE_ID = data['CVE_Items'][i]['cve']['problemtype']['problemtype_data'][0]['description'][0]['value'] #CVE Type
                        # Counting CWEs with values NVD-CWE-Other and NVD-CWE-noinfo
                        if CWE_ID == 'NVD-CWE-Other':
                            M_CWE_ID_Other+=1
                            CWE_Desc, CWE_Plat, CWE_Af_Res = "N/A", "N/A", "N/A"
                            CWE_Consequences, CWE_Mitigations = {}, {}
                            M_CWE_Desc, M_CWE_Plat, M_CWE_Af_Res, M_CWE_Consequences, M_CWE_Mitigations = (M_CWE_Desc+1), (M_CWE_Plat+1), (M_CWE_Af_Res+1), (M_CWE_Consequences+1), (M_CWE_Mitigations+1) # increment counters 
                        elif CWE_ID == 'NVD-CWE-noinfo':
                            M_CWE_ID_Other2+=1
                            CWE_Desc, CWE_Plat, CWE_Af_Res = "N/A", "N/A", "N/A"
                            CWE_Consequences, CWE_Mitigations = {}, {}
                            M_CWE_Desc, M_CWE_Plat, M_CWE_Af_Res, M_CWE_Consequences, M_CWE_Mitigations = (M_CWE_Desc+1), (M_CWE_Plat+1), (M_CWE_Af_Res+1), (M_CWE_Consequences+1), (M_CWE_Mitigations+1) # increment counters 

                        else: # when CWE-ID is an actual number
                            CWE_Desc, CWE_Plat, CWE_Af_Res, CWE_Consequences, CWE_Mitigations = fetch_cwe_data(CWE_ID) # fetch CWE Data from CWE File.
                            if CWE_Desc=='-': CWE_Desc="N/A"
                            if CWE_Plat=='-': CWE_Plat="N/A"
                            if CWE_Af_Res=='-': CWE_Af_Res="N/A"
                            
                    except IndexError: # Some CVEs are missing Vuln Type and Lang values.
                        # CWE_ID = "N/A" # this is not required. 
                        M_CWE_ID+=1

                    try: # Field: severity
                        CVE_Sev = data['CVE_Items'][i]['impact']['baseMetricV2']['severity']
                    except KeyError: # Reserved CVEs will have this field value missing
                        CVE_Sev = "N/A"
                        M_CVE_Sev+=1

                    try: # Field: V2 Score
                        CVSS_V2 = data['CVE_Items'][i]['impact']['baseMetricV2']['cvssV2']['baseScore']
                    except KeyError: # Reserved CVEs will have this field value missing
                        CVSS_V2 = -1
                        M_CVSS_V2+=1
                        
                    try: # Field: CVSS Score V3, not all CVEs have CVSS v3 Scores so we keep rest as zero.
                        CVSS_V3 = data['CVE_Items'][i]['impact']['baseMetricV3']['cvssV3']['baseScore']
                    except KeyError:
                        CVSS_V3 = -1
                        M_CVSS_V3+=1

                    try: # Field: accessVector, not all CVEs have CVSS v3 Scores so we keep rest as zero.
                        CVE_VAV = data['CVE_Items'][i]['impact']['baseMetricV2']['cvssV2']['accessVector']
                    except KeyError:
                        CVE_VAV = "N/A"
                        M_CVE_VAV+=1

                    try: # Field: userInteractionRequired, not all CVEs have CVSS v3 Scores so we keep rest as zero.
                        CVE_UIR = data['CVE_Items'][i]['impact']['baseMetricV2']['userInteractionRequired']
                    except KeyError:
                        CVE_UIR = "N/A"
                        M_CVE_UIR+=1    

                    try: # Field: publishedDate, not all CVEs have CVSS v3 Scores so we keep rest as zero.
                        CVE_PD = data['CVE_Items'][i]['publishedDate']
                    except KeyError:
                        CVE_PD = "N/A"
                        M_CVE_PD+=1 

                    try: # Field: lastModifiedDate, not all CVEs have CVSS v3 Scores so we keep rest as zero.
                        CVE_MD = data['CVE_Items'][i]['lastModifiedDate']
                    except KeyError:
                        CVE_MD = "N/A"
                        M_CVE_MD+=1 
                    
                    try:
                        # Write row to the output CVS file...
                        # enable next line for debugging...
                        # print(CVE_ID, end=' ')
                        try:
                            writer.writerow([CVE_ID, CVE_Lang, CWE_ID, CWE_Desc, CWE_Plat, CWE_Af_Res, json.dumps(CWE_Consequences), json.dumps(CWE_Mitigations), CVE_Sev, CVSS_V2, 
                                CVSS_V3, CVE_VAV, CVE_UIR, json.dumps(CVE_OS), json.dumps(CVE_SW), json.dumps(CVE_Ports), 
                                CVE_PD, CVE_MD, CVE_Desc, url_tag_dict])
                            # create DB Doc
                            doc = {"_id": CVE_ID, "lang": CVE_Lang, "cwe_id": CWE_ID,"cwe_desc": CWE_Desc, 
                                   "cwe_plat": CWE_Plat, "cwe_af_res": CWE_Af_Res, "cwe_consequences": CWE_Consequences,
                                   "cwe_mitigations": CWE_Mitigations, "severity": CVE_Sev, "cvss_v2": CVSS_V2, 
                                   "cvss_v3": CVSS_V3, "vul_access_vector": CVE_VAV,  "user_int_req": CVE_UIR, "os": CVE_OS, 
                                   "sw": CVE_SW,"ports": CVE_Ports, "published_date": CVE_PD, "modified_date": CVE_MD, "description": CVE_Desc, 
                                   "url_and_tags": url_tag_dict # json.dumps(url_tag_dict) 
                            } # print('Doc:\n', json.dumps(doc, indent=4))
                            
                            # Insert or update the document in the database
                            func.db.put(doc)

                        except UnicodeEncodeError: # may occure for description field
                            writer.writerow([CVE_ID, CVE_Lang, CWE_ID, CWE_Desc, CWE_Plat, CWE_Af_Res, json.dumps(CWE_Consequences), json.dumps(CWE_Mitigations), CVE_Sev, CVSS_V2, 
                                CVSS_V3, CVE_VAV, CVE_UIR, json.dumps(CVE_OS), json.dumps(CVE_SW), json.dumps(CVE_Ports), 
                                CVE_PD, CVE_MD, CVE_Desc.encode("utf-8"), url_tag_dict])
                            unicode_error+=1
                            # create DB Doc
                            doc = {"_id": CVE_ID, "lang": CVE_Lang, "cwe_id": CWE_ID, "cwe_desc": CWE_Desc, 
                                   "cwe_plat": CWE_Plat, "cwe_af_res": CWE_Af_Res, "cwe_consequences": CWE_Consequences,
                                   "cwe_mitigations": CWE_Mitigations, "severity": CVE_Sev, "cvss_v2": CVSS_V2, 
                                   "cvss_v3": CVSS_V3, "vul_access_vector": CVE_VAV, "user_int_req": CVE_UIR, "os": CVE_OS, 
                                   "sw": CVE_SW, "ports": CVE_Ports, "published_date": CVE_PD, "modified_date": CVE_MD, 
                                   "description": CVE_Desc.encode("utf-8"), "url_and_tags": url_tag_dict
                            } # print('Doc:\n', json.dumps(doc, indent=4))
                            print('Doc:\n', json.dumps(doc, indent=4))
                            # Insert or update the document in the database
                            func.db.put(doc)
                    except TypeError:
                        print('Type error with doc', CVE_ID)

    print('Insertion complete.')
    print('\nTotal number of vulnerability records found:',vul_info_counter, '\nVulnerabilities with missing information:')
    print('Lang:\t\t'+str(M_CVE_Lang)+'\n\nCWE-ID:\t\t'+str(M_CWE_ID)+'\nNVD-CWE-Other\t'+str(M_CWE_ID_Other)+
          '\nNVD-CWE-noinfo:\t'+str(M_CWE_ID_Other2)+'\nTotal CWE:\t'+str(M_CWE_ID+M_CWE_ID_Other+M_CWE_ID_Other2)+
          '\nMitigation:\t'+str(M_CWE_Mitigations)+'\nConsequences:\t'+str(M_CWE_Consequences)+
          '\n\nSeverity:\t'+str(M_CVE_Sev)+'\nCVSS_V2:\t'+str(M_CVSS_V2)+
          '\nCVSS_V3:\t'+str(M_CVSS_V3)+'\nVul_access:\t'+str(M_CVE_VAV)+'\nUserIntReq:\t'+str(M_CVE_UIR)+'\nPublishDate:\t'+
          str(M_CVE_PD)+'\nModifiedDate:\t'+str(M_CVE_MD)+'\nUnicodeError:\t'+str(unicode_error))
    print('\nCWE_Desc:\t'+str(M_CWE_Desc)+'\nCWE_Plat:\t'+str(M_CWE_Plat)+'\nCWE_Af_Res:\t'+str(M_CWE_Af_Res)+
          '\nM_OS:\t\t'+str(M_OS)+'\nM_SW:\t\t'+str(M_SW)+'\nM_Ports:\t'+str(M_Ports))
    print('\n')

# **************************************************************
# 3. Fetch OS, SW Versions, and Port Numbers from given CVE Description...
nlp = spacy.load('en_core_web_sm')
def fetch_keywords(text):
    # extract port numbers from text
    port_list = find_ports_from_text(text)
    
    # extract sw and os related terms.
    match_list = []
    patterns = [
                [{'POS': 'PROPN'}, {'POS': {"IN": ["PROPN","NUM", "X", "VERB"]}}],
                [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': {"IN": ["NUM","X","VERB"]}}],    
               ]
    matcher = Matcher(nlp.vocab)
    matcher.add("PROPN-PROPN-NUM", None, patterns[0])
    matcher.add("PROPN-PROPN-VERB", None, patterns[1])
    doc = nlp(text)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        match_list.append(span.text)
    
    # Remove duplicates from the list
    match_list = list(dict.fromkeys(match_list))
    
    # Remove subsets from the list ['Microsoft Windows 2000', 'Windows 2000']
    for m in match_list:
        for n in match_list:
            if (len(m) > len(n)) and (set(n).issubset(set(m))):
                match_list.remove(n)

    # Extract OS from the list...
    #ToDo: create an OS List and reference that list instead of these if statements.
    os_list = [idx for idx in match_list if 
            idx.lower().startswith('Microsoft'.lower()) or idx.lower().startswith('Windows'.lower()) or 
            idx.lower().startswith('Ubuntu'.lower()) or idx.lower().startswith('Linux'.lower()) or 
            idx.lower().startswith('Apple'.lower()) or idx.lower().startswith('macOS'.lower()) or 
            idx.lower().startswith('RedHat'.lower()) or idx.lower().startswith('Red Hat'.lower()) or 
            idx.lower().startswith('CentOS'.lower()) or idx.lower().startswith('Fedora'.lower()) or 
            idx.lower().startswith('openSUSE'.lower())
          ] 

    # Extract SW names from the list...
    sw_list = []
    for elem in match_list:
        if elem not in os_list:
            sw_list.append(elem) 
    return os_list, sw_list, port_list


# Port extractor from string... 
def find_ports_from_text(text):
    port_list = []
    nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
    doc = nlp(text)
    # ? (0/1) + (1+) * (0+)
    port_pattern = re.compile(r"[Pp](ort)[s]? [:]?(\d+)?((\d+|less than | and |, |,|/|/ | / | |-)?(\d+))*") #  

    for match in re.finditer(port_pattern, doc.text):
        start, end = match.span()
        # if text is only 'port ' then discard it
        if doc.text[start:end].lower() != 'port ' or doc.text[start:end].lower() != 'ports ': 
            port_list.append(doc.text[start:end])
        span = doc.char_span(start, end)
    # print('\nPorts related keywords:\n', port_list)
    return port_list


# Referencing CWE File for fetching CWE Descriptions and related data...
# Extracting 'CWE_Desc', 'CWE_Plat', 'CWE_Af_Res'
def fetch_cwe_data(given_cwe_id):  
    result = cwe_data.loc[cwe_data['_id'] == given_cwe_id] #_id (CWE-ID) on cwe_combined file
    CWE_Description = result.iloc[0,1]
    CWE_App_Plat = result.iloc[0,8]
    CWE_Aff_Res = result.iloc[0,19]
    CWE_Consequences = process_consequences(result.iloc[0,14]) # process received data and make a dictionary
    CWE_Mitigations = process_mitigations(result.iloc[0,16]) # process received data and make a dictionary
    return CWE_Description, CWE_App_Plat, CWE_Aff_Res, CWE_Consequences, CWE_Mitigations


# process consequences text and generate a dictionary out of it.
def process_consequences(con):
    data = {}
    scope_list = con.split('::SCOPE:') # Split by ::SCORE:
    scope_list = [x for x in scope_list if x] # remove blank items ('')
    scope_list = [x.replace('::', '') for x in scope_list] # remove :: from end of items

    for i in scope_list: # loop through scopes  
        sub_data={}
        kw = i.split(':')[0] # first word of sentence ''
        j = i.replace(kw, '') # remove keyword from the item
        
        scope_sub_items = re.split(r'(:SCOPE:|:IMPACT:|:NOTE:)',j) 
        scope_sub_items = [x for x in scope_sub_items if x] # remove blank items ('')
        for item in range(0, len(scope_sub_items), 2): # concat following items 1*2, 3*4, etc.
            new_form = scope_sub_items[item].replace(':', '')+'*'+scope_sub_items[item+1]
            c = new_form.split('*')
            if c[0] not in sub_data:
                sub_data[c[0]] = []
            sub_data[c[0]].append(c[1])
        
        if kw not in data:
            data[kw]=sub_data
    return data
 

# process mitigation text and generate a dictionary out of it.
def process_mitigations(mit):
    data = {}
    scope_list = mit.split('::PHASE:') # Split by ::SCORE:
    scope_list = [x for x in scope_list if x] # remove blank items ('')
    scope_list = [x.replace('::', '') for x in scope_list] # remove :: from end of items

    for i in scope_list: # loop through scopes  
        sub_data={}
        kw = i.split(':')[0] # first word of sentence ''
        j = i.replace(kw, '') # remove keyword from the item
        
        scope_sub_items = re.split(r'(:STRATEGY:|:DESCRIPTION:|:EFFECTIVENESS:)',j) 
        scope_sub_items = [x for x in scope_sub_items if x] # remove blank items ('')
        for item in range(0, len(scope_sub_items), 2): # concat following items 1*2, 3*4, etc.
            new_form = scope_sub_items[item].replace(':', '')+'*'+scope_sub_items[item+1]
            c = new_form.split('*')
            if c[0] not in sub_data:
                sub_data[c[0]] = []

            sub_data[c[0]].append(c[1])

        if kw not in data: # if key not present in data
            data[kw]=[]
            data[kw].append(sub_data)

        else: # if key present
            for k, v in sub_data.items(): # add under same keys
                if k in data[kw][0]: # if key found
                    if v not in data[kw][0][k]: # if value not found, add it under value list
                        data[kw][0][k].extend(v)
                else: 
                    data[kw][0][k] = v

    d = {k:v[0] for k,v in data.items()} # structure fix, list to dict.
    return d

print('*** CyVID Dataset Preparation v5 ***\n')
# time the execution
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time:", current_time)
startTime = datetime.now() # start timer
spinner.start()

# Step 1: Check and remove existing files and folders
func.check_files(analysis_path)

# Step 2, read JSON files, summarize data and extract keywords...
read_and_summarize()
# Step 3 and 4 will execute based on current CWE-ID and CVE-Description, therefore it will run within Step 2.

print('\nData collected.\nFile(s) saved at '+analysis_path+'/. and Database: '+db_name+' created.')
print("Execution time: "+str(datetime.now() - startTime))
spinner.stop()

*** CyVID Dataset Preparation v5 ***

Start Time: 14:10:49
|Directory analysis already exists.
Removing old files... Done.

Reading, summarizing, and writing NVD JSON Data...
Server status: (3.1.1) up and running!
|Insertion complete.

Total number of vulnerability records found: 166759 
Vulnerabilities with missing information:
Lang:		9783

CWE-ID:		23873
NVD-CWE-Other	26646
NVD-CWE-noinfo:	15308
Total CWE:	65827
Mitigation:	41954
Consequences:	41954

Severity:	9829
CVSS_V2:	9829
CVSS_V3:	83171
Vul_access:	9829
UserIntReq:	10854
PublishDate:	0
ModifiedDate:	0
UnicodeError:	0

CWE_Desc:	41954
CWE_Plat:	41954
CWE_Af_Res:	41954
M_OS:		0
M_SW:		0
M_Ports:	0



Data collected.
File(s) saved at analysis/. and Database: cyvia_dataset created.
Execution time: 0:21:04.759101


In [ ]:
As of 7/21/21 166759 CVEs.